In [1]:
import sys
sys.path.append('../app')
import cipher

from base64 import b64encode, b64decode
import bcrypt
import csv
import MySQLdb
from tqdm.notebook import tqdm

In [21]:
## originally used to add all users to DB

# fn = '../data/unicorn-keys-2.csv'

# with tunnel:
#     db = MySQLdb.connect(
#         user = os.environ.get('MYSQL_DB_USER'),
#         password = os.environ.get('MYSQL_DB_PASSWORD'),
#         database = os.environ.get('MYSQL_DB_DATABASE'),
#         host = '127.0.0.1',
#         port = BIND_PORT
#     )
#     cur = db.cursor()

#     nrows = 0
#     with open(fn) as f:
#         for _ in csv.reader(f):
#             nrows += 1
    
#     with open(fn) as f:   
#         for row in tqdm(csv.reader(f), total=nrows):
#             name = row[0]
#             fullname = row[1]
#             pw = fullname + '$' + row[2]
#             keyhash = bcrypt.hashpw(row[2].encode(), bcrypt.gensalt()).decode()
            
#             cur.execute(
#                 "insert into persons (name, fullname, secret_key_hash) values (%s, %s, %s)",
#                 (name, fullname, keyhash)
#             )

#     db.commit()
#     cur.close()
#     db.close()

In [6]:
### originally used to add x25519 keys for all users to DB

fn = '../data/unicorn-keys.csv'

db = MySQLdb.connect(
    user = os.environ.get('MYSQL_DB_USER'),
    password = os.environ.get('MYSQL_DB_PASSWORD'),
    database = os.environ.get('MYSQL_DB_DATABASE'),
    host = os.environ.get('MYSQL_DB_HOST'),
    port = 3306
)
cur = db.cursor()

nrows = 0
with open(fn) as f:
    for _ in csv.reader(f):
        nrows += 1

with open(fn) as f:
    for row in tqdm(csv.reader(f), total=nrows):
        pid, pw = cipher.user_info_from_secret(row[2])
        priv, pub = cipher.generate_key_pair()
        public_key = cipher.serialize_public_key(pub)
        encrypted_private_key = cipher.encrypt_private_key(priv, pw)
        keyhash = bcrypt.hashpw(pw, bcrypt.gensalt()).decode()

        cur.execute(
            "update persons set secret_key_hash = %s where id = %s",
            (keyhash, pid)
        )
        cur.execute(
            "update persons set public_key = %s where id = %s",
            (public_key, pid)
        )
        cur.execute(
            "update persons set encrypted_private_key = %s where id = %s",
            (encrypted_private_key, pid)
        )

db.commit()
cur.close()
db.close()

  0%|          | 0/63 [00:00<?, ?it/s]

In [11]:
# nuke and reset keys for the test user

with MySQLdb.connect(
    user = os.environ.get('MYSQL_DB_USER'),
    password = os.environ.get('MYSQL_DB_PASSWORD'),
    database = os.environ.get('MYSQL_DB_DATABASE'),
    host = os.environ.get('MYSQL_DB_HOST'),
    port = 3306
) as db, db.cursor() as cur:
    pid, pw = cipher.user_info_from_secret('an_extremely_secret_secret')
    priv, pub = cipher.generate_key_pair()
    public_key = cipher.serialize_public_key(pub)
    encrypted_private_key = cipher.encrypt_private_key(priv, pw)
    keyhash = bcrypt.hashpw(pw, bcrypt.gensalt()).decode()
    
    cur.execute(
        "update persons set secret_key_hash = %s where id = %s",
        (keyhash, pid)
    )
    cur.execute(
        "update persons set public_key = %s where id = %s",
        (public_key, pid)
    )
    cur.execute(
        "update persons set encrypted_private_key = %s where id = %s",
        (encrypted_private_key, pid)
    )

    db.commit()